In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import root_scalar

In [8]:
def fibonaccigrid_sampling(n):
    phi = (1 + np.sqrt(5)) / 2  
    
    # Initialize arrays for Cartesian coordinates
    points = np.zeros((n, 4))
    
    for k in range(n):
        #Compute angles using the index k and n points
        theta = np.arccos(1 - 2 * (k + 0.5) / n)  # Range [0, π]
        phi_k = 2 * np.pi * (k / phi**2)          # Range [0, 2π], scaled by golden ratio squared
        psi_k = 2 * np.pi * (k / phi**3)          # Additional rotation for 3rd angle, golden ratio cubed
        
        #Convert spherical coordinates to Cartesian coordinates in 4D
        points[k, 0] = np.cos(theta)
        points[k, 1] = np.sin(theta) * np.cos(phi_k)
        points[k, 2] = np.sin(theta) * np.sin(phi_k) * np.cos(psi_k)
        points[k, 3] = np.sin(theta) * np.sin(phi_k) * np.sin(psi_k)
    
    return points

In [9]:
def vertical_projection(v, p):
    """
    Compute the vertical projection of v at p in real coordinates.
    
    Parameters:
        v (np.ndarray): Tangent vector in R^4.
        p (np.ndarray): Base point in R^4.

    Returns:
        np.ndarray: Vertical component of v.
    """
    # Vertical vector field at p (real-valued)
    vertical_direction = np.array([-p[1], p[0], -p[3], p[2]])  
    vertical_direction /= np.linalg.norm(vertical_direction)
    projection = np.dot(v, vertical_direction) * vertical_direction
    return projection

def horizontal_projection(v, p):
    """
    Project v into the horizontal space at p.
    
    Parameters:
        v (np.ndarray): Tangent vector in R^4.
        p (np.ndarray): Base point in R^4.

    Returns:
        np.ndarray: Horizontal component of v.
    """
    vert_proj = vertical_projection(v, p)
    return v - vert_proj


In [10]:
def geodesic_error(params, p, q):
    """
    Compute the error for the sub-Riemannian geodesic.
    
    Parameters:
        params (np.ndarray): Flattened array of v (4D) and s1 (scalar).
        p (np.ndarray): Starting point in R^4.
        q (np.ndarray): Target point in R^4.

    Returns:
        float: Squared norm of the difference between the geodesic and q.
    """
    v = params[:-1]  # Velocity vector
    s1 = params[-1]  # Time parameter
    
    # Normalize v to lie in the horizontal space
    v = horizontal_projection(v, p)
    norm_v = np.linalg.norm(v)

    if norm_v < 1e-10:
        return np.inf  # Prevent division by zero
    
    # Riemannian geodesic
    gamma_R = p * np.cos(norm_v * s1) + (v / norm_v) * np.sin(norm_v * s1)
    
    # Compute error
    return np.linalg.norm(gamma_R - q)**2

In [11]:
def find_subriemannian_geodesic(p, q):
    """
    Find the initial velocity v and parameter s1 such that the sub-Riemannian geodesic passes through q.
    
    Parameters:
        p (np.ndarray): Starting point in R^4.
        q (np.ndarray): Target point in R^4.

    Returns:
        tuple: (v, s1) where v is the initial velocity and s1 is the parameter 
        at which the geodesic meets q.
    """
    initial_v = np.random.randn(4)  
    initial_v = horizontal_projection(initial_v, p)  
    initial_s = 1.0  
    initial_params = np.concatenate([initial_v, [initial_s]])

    # Define constraints: v must be orthogonal to p to be in TpS^3
    constraints = {
        'type': 'eq',
        'fun': lambda params: np.dot(p, params[:-1]) 
    }

    # Minimize the geodesic error
    result = minimize(
        geodesic_error,
        initial_params,
        args=(p, q),
        constraints=constraints,
        bounds=[(None, None)] * 4 + [(0, 2 * np.pi)]  
    )

    if result.success:
        v = result.x[:-1]  
        s1 = result.x[-1] 
        return v, s1
    else:
        raise ValueError("Optimization failed to find v and s1.")

In [12]:
points = fibonaccigrid_sampling(10)
for p in points:
    for q in points:
        v, s1 = find_subriemannian_geodesic(p, q)
        print("Initial point: ", p)
        print("Target point: ", q)
        print("Initial velocity: ", v)
        print("Parameter s1: ", s1)      

Initial point:  [0.9        0.43588989 0.         0.        ]
Target point:  [0.9        0.43588989 0.         0.        ]
Initial velocity:  [ 3.08306436e-10 -2.10449685e-10  5.25976548e-01  7.28757824e-01]
Parameter s1:  0.0
Initial point:  [0.9        0.43588989 0.         0.        ]
Target point:  [ 0.7        -0.52658671  0.04217387  0.48054948]
Initial velocity:  [-7.42073368e-08  1.53218976e-07  7.19040724e-02  8.18950413e-01]
Parameter s1:  1.068028820813891
Initial point:  [0.9        0.43588989 0.         0.        ]
Target point:  [ 0.5         0.0757129   0.84952161 -0.15026841]
Initial velocity:  [-3.24762022e-09  6.70549657e-09  1.14264670e+00 -2.02196531e-01]
Parameter s1:  0.9136530236386986
Initial point:  [0.9        0.43588989 0.         0.        ]
Target point:  [ 0.3         0.58041368 -0.19653263 -0.73109157]
Initial velocity:  [-7.60240277e-09  1.56969973e-08 -1.12915111e-01 -4.19772239e-01]
Parameter s1:  2.2230964579863324
Initial point:  [0.9        0.435889

In [ ]:
def arccot(x):
    return np.arctan(1 / x) if x > 0 else np.pi / 2

In [ ]:
def distance(p,q):
    """
    Compute the length of the sub-Riemannian geodesic from p to q.
    Parameters:
        p (np.ndarray): Starting point in R^4.
        q (np.ndarray): Target point in R^4.
    Returns:
        float: Length of the geodesic.
    """
    v,s = find_subriemannian_geodesic(p,q)
    vf = vertical_projection(v,p)
    u = abs(vf)/np.linalg.norm(v)
    rho = np.linalg.norm(v)
    #is sgn sign or genealized signus function?
    theta1 = np.sign(np.cos(rho))
    theta2 = np.sign(np.sin(rho))
    #I think i need to give back already z1 and z2 as they want it.
    #geodesic evaluated at the endpoint q
    z1 = np.exp(-1j*u*rho*s)*(np.cos(rho*s)+1j*np.sin(rho*s))
    z2 = 0
    def target_function(u):
        term1 = arccot(np.sqrt(abs(z1)**2 - u**2) / (u * abs(z2)))
        term2 = u * arccot(np.sqrt(abs(z1)**2 - u**2) / abs(z2))
        return term1 - term2 - np.sign(np.sin(rho))
    if theta1>0 and theta2>0:
        result = root_scalar(target_function, method='brentq', bracket=(p, q))
        if result.converged:
            u0 = result.root
            d = np.sqrt(1-u0**2)*arccot(np.sqrt(abs(z1)**2 - u0**2) / abs(z2))
        else:
            print("Root-finding did not converge.")

a bit of testing, still doesnt work

In [ ]:
points = fibonaccigrid_sampling(10)
for p in points:
    for q in points:
        v, s1 = find_subriemannian_geodesic(p, q)
        d = distance(p, q)
        print("Initial point: ", p)
        print("Target point: ", q)
        print("Initial velocity: ", v)
        print("Parameter s1: ", s1)
        print("Distance: ", d)

COMPUTE DISTANCE MATRIX

In [ ]:
def a_matrix(points):
    A = np.zeros((len(points), len(points)))
    for i, p in enumerate(points):
        for j, q in enumerate(points):
            d = distance(p, q)
            A[i, j] = -0.5 * d**2
    return A

In [ ]:
def b_matrix(points):
    A = a_matrix(points)
    H = np.identity(len(points)) - np.ones((len(points), len(points))) / len(points)
    #@ operator for matrix multiplication
    B = H@A@H
    return B